In [127]:
import pandas as pd

In [128]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [129]:
PATH = "https://storage.yandexcloud.net/data-ice-lake-04/messager-data/analytics/tmp/wide-dataframe/part-00000-8202347f-33da-4e2f-9e57-b6cded00abdb-c000.snappy.parquet"

In [130]:
df = pd.read_parquet(path=PATH)

df.head()

,admins,channel_id,datetime,media_type,media_src,message,message_channel_to,message_from,message_group,message_id,message_to,message_ts,reaction_from,reaction_type,subscription_channel,subscription_user,tags,user
0,None,NaN,NaT,None,None,how can i do that,NaN,112012,NaN,615,149488.0,2021-05-14 16:45:46,None,None,NaN,None,None,None
1,None,NaN,NaT,None,None,hi all. i am using skype to talk with my frie...,NaN,99752,NaN,645,149488.0,2021-05-14 20:02:56,None,None,NaN,None,None,None
2,None,NaN,NaT,None,None,"you can, you'd just have to take away some spa...",NaN,4226,NaN,929,7964.0,2021-05-14 11:06:52,None,None,NaN,None,None,None
3,None,NaN,NaT,None,None,I may just be retarded,NaN,32481,NaN,2080,149488.0,2021-05-14 23:10:39,None,None,NaN,None,None,None
4,None,NaN,NaT,None,None,kill -9 PID,NaN,123432,NaN,2785,124190.0,2021-05-14 04:27:14,None,None,NaN,None,None,None


In [131]:
df.shape

(109763, 18)

In [132]:
df = df[df['message_channel_to'].notna()]
df.shape

(20885, 18)

In [133]:
df.tags.isna().sum()

532

In [134]:
df = df[['message_from', 'message_id','tags']]

In [135]:
df = df.explode('tags').rename(columns={'message_from': 'user', 'tags':'tag'})

In [136]:
df.shape

(94948, 3)

In [137]:
df = df.groupby(['user', 'tag'])['message_id'].count().reset_index()

In [138]:
df.head(20)

,user,tag,message_id
0,5,MySQL,1
1,5,accellion,1
2,5,enterprise,1
3,5,file-sharing,1
4,5,funding,1
5,14,Clojure,1
6,14,Копирайт,1
7,14,Разработка веб-сайтов,1
8,14,Функциональное программирование,1
9,26,Python,1


In [139]:
df.shape[0]

93883

In [140]:
df[df.message_id > 1].shape[0] 

515

In [141]:
df = df.rename(columns={'message_id':'tag_cnt'})

In [142]:
df[df.tag_cnt > 1].head(5)

,user,tag,tag_cnt
363,680,ios,2
546,1015,Чулан,2
659,1222,google,2
750,1409,PHP,2
1277,2462,Чулан,2


In [143]:
df = df.sort_values(by=['user', 'tag_cnt', 'tag'], ascending=[True, False, True])

In [144]:
df['tag_rank'] = df.groupby(['user'])['user'].rank(method='first')

In [145]:
df = df[df.tag_rank <= 3]

In [146]:
mapper = {1: "tag_top_1", 2: "tag_top_2", 3: "tag_top_3"}

In [147]:
df['tag_rank'] = df['tag_rank'].map(mapper)

In [148]:
df.head(10)

,user,tag,tag_cnt,tag_rank
0,5,MySQL,1,tag_top_1
1,5,accellion,1,tag_top_2
2,5,enterprise,1,tag_top_3
5,14,Clojure,1,tag_top_1
6,14,Копирайт,1,tag_top_2
7,14,Разработка веб-сайтов,1,tag_top_3
9,26,Python,1,tag_top_1
10,26,cisneros,1,tag_top_2
11,26,furniture,1,tag_top_3
18,54,conversocial,1,tag_top_1


In [149]:
df[df.user==70].head(10)

,user,tag,tag_cnt,tag_rank
34,70,Транспорт,1,tag_top_1


In [150]:
df = df.pivot(index='user', columns='tag_rank', values='tag')

In [151]:
df.head()

tag_rank,tag_top_1,tag_top_2,tag_top_3
user,,,
5,MySQL,accellion,enterprise
14,Clojure,Копирайт,Разработка веб-сайтов
26,Python,cisneros,furniture
54,conversocial,joshua-march,octopus-investments
55,europe,forget-me,privacy


In [159]:
df.reset_index(level=0).head()

tag_rank,user,tag_top_1,tag_top_2,tag_top_3
0,5,MySQL,accellion,enterprise
1,14,Clojure,Копирайт,Разработка веб-сайтов
2,26,Python,cisneros,furniture
3,54,conversocial,joshua-march,octopus-investments
4,55,europe,forget-me,privacy


In [116]:
del df.tag_rank

tag_rank,user,tag_top_1,tag_top_2,tag_top_3
0,5,MySQL,accellion,enterprise
1,14,Clojure,Копирайт,Разработка веб-сайтов
2,26,Python,cisneros,furniture
3,54,conversocial,joshua-march,octopus-investments
4,55,europe,forget-me,privacy


In [126]:
df.head()

tag_rank,user,tag_top_1,tag_top_2,tag_top_3
0,5,MySQL,accellion,enterprise
1,14,Clojure,Копирайт,Разработка веб-сайтов
2,26,Python,cisneros,furniture
3,54,conversocial,joshua-march,octopus-investments
4,55,europe,forget-me,privacy
